# Gemma3-4b fine-tune for food and beverage label OCR-VQA

In [2]:
#%pip install torch tensorboard
#%pip install transformers datasets accelerate evaluate trl protobuf sentencepiece
#%pip install flash-attn
#%pip install accelerate peft

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
  Using cached flash_attn-2.8.2.tar.gz (8.2 MB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [21 lines of output]
      /tmp/pip-install-7e6fg1wa/flash-attn_25a242632d1745688fee32b11463a655/setup.py:106: UserWarning: flash_attn was requested, but nvcc was not found.  Are you sure your environment has nvcc available?  If you're installing within a container from https://hub.docker.com/r/pytorch/pytorch, only images whose names contain 'devel' will provide nvcc.
        warnings.warn(
      Traceback (most recent c

In [3]:
#%pip install tf-keras

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os

In [5]:
hf_token = os.getenv('HF_TOKEN')

In [6]:
#%env HF_HOME=/local/scratch/crowelenn-aiml339/

env: HF_HOME=/local/scratch/crowelenn-aiml339/


In [7]:
base_model = "google/gemma-3-4b-it"
checkpoint_dir = "/local/scratch/crowelenn-aiml339/checkpoints"
learning_rate = 5e-5

Tensorboard setup

In [8]:
#%pip install tensorboard

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [9]:
system_message = "You are a quality control robot responsible for monitoring the quality of supplement labels."
user_prompt = """Using primarily the text contained in the attached label supplement image, answer the list of questions in the <QUESTIONS> tags.
Answer concisely in a JSON format with no preamble, allowing the response to easily be parsed. An example response would be:
{
  "brand": "label supplelments co",
  "contents": 120
}

<QUESTIONS>
"""

In [10]:
def format_data(sample):
    return {
        "messages": [
            {
                "role": "system",
                "content": [{"type": "text", "text": system_message}],
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": sample["questions"]
                        #"text": user_prompt.format(
                        #    questions=sample["questions"]
                        #),
                    },
                    {
                        "type": "image",
                        "image": sample["image"],
                    },
                ],
            },
            {
                "role": "assistant",
                "content": [{"type": "text", "text": sample["answers"]}],
            },
        ],
    }

In [11]:
import json, random
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms

class OCRVQADataset(Dataset):
    def __init__(self, jsonl_file, transform=None, min_q=1, max_q=4):
        with open(jsonl_file, 'r') as f:
            self.samples = [json.loads(line) for line in f]
        self.transform = transform or transforms.ToTensor()
        self.min_q = min_q
        self.max_q = max_q

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]

        # Keep as PIL so process_vision_info works later
        image_path = "AIML339/supplements_small/images/" + sample["image"]
        image = Image.open(image_path).convert("RGB")

        qa_pairs = sample["qas"]
        k = random.randint(self.min_q, min(self.max_q, len(qa_pairs)))
        chosen_pairs = random.sample(qa_pairs, k)

        questions_str = (
            user_prompt
            + "; ".join(f"Question: {p['q']} This corresponds to JSON key {p['k']}" for p in chosen_pairs)
            + "</QUESTIONS>"
        )
        answers_dict = {p['k']: p['a'] for p in chosen_pairs}
        answers_str = json.dumps(answers_dict, ensure_ascii=False)

        return {
            "image": image,  # PIL
            "questions": questions_str,
            "answers": answers_str
        }

In [12]:
def process_vision_info(messages: list[dict]) -> list[Image.Image]:
    image_inputs = []
    for msg in messages:
        content = msg.get("content", [])
        if not isinstance(content, list):
            content = [content]

        for element in content:
            if isinstance(element, dict) and (
                "image" in element or element.get("type") == "image"
            ):
                img = element.get("image", element)
                if isinstance(img, str):
                    img = Image.open(img).convert("RGB")
                elif not isinstance(img, Image.Image):
                    raise ValueError(f"Unsupported image type: {type(img)}")
                image_inputs.append(img)
    return image_inputs


In [13]:
dataset_path = r'AIML339/supplements_small/output.jsonl'
dataset_obj = OCRVQADataset(dataset_path)

In [14]:
import torch
from torch.utils.data import random_split

# Set split sizes
total_size = len(dataset_obj)
train_size = int(0.8 * total_size)
val_size = int(0.1 * total_size)
test_size = total_size - train_size - val_size

# Use a generator with a manual seed for reproducibility
generator = torch.Generator().manual_seed(42)

In [15]:
train_dataset, val_dataset, test_dataset = random_split(
    dataset_obj, [train_size, val_size, test_size], generator=generator
)

train_dataset_fmt = [format_data(sample) for sample in train_dataset]

print(train_dataset_fmt[100])

{'messages': [{'role': 'system', 'content': [{'type': 'text', 'text': 'You are a quality control robot responsible for monitoring the quality of supplement labels.'}]}, {'role': 'user', 'content': [{'type': 'text', 'text': 'Using primarily the text contained in the attached label supplement image, answer the list of questions in the <QUESTIONS> tags.\nAnswer concisely in a JSON format with no preamble, allowing the response to easily be parsed. An example response would be:\n{\n  "brand": "label supplelments co",\n  "contents": 120\n}\n\n<QUESTIONS>\nQuestion: What is the name of the brand? This corresponds to JSON key brand; Question: What is the numeric component of the product contents? This corresponds to JSON key contents</QUESTIONS>'}, {'type': 'image', 'image': <PIL.Image.Image image mode=RGB size=1339x1150 at 0x558FD8D08400>}]}, {'role': 'assistant', 'content': [{'type': 'text', 'text': '{"brand": "BNRG Bionutritional Research Group", "contents": "1"}'}]}]}


In [16]:
from transformers import AutoProcessor, AutoModelForImageTextToText
from peft import LoraConfig, get_peft_model
from accelerate import Accelerator
from torch.utils.tensorboard import SummaryWriter

accelerator = Accelerator(mixed_precision="bf16")
writer = SummaryWriter(log_dir="./logs")

# Hugging Face model id
model_id = "google/gemma-3-4b-it" # or `google/gemma-3-12b-pt`, `google/gemma-3-27-pt`

# Check if GPU benefits from bfloat16
#if torch.cuda.get_device_capability()[0] < 8:
#    raise ValueError("GPU does not support bfloat16, please use a GPU that supports bfloat16.")

# Define model init arguments
#model_kwargs = dict(
#    attn_implementation="eager", # Use "flash_attention_2" when running on Ampere or newer GPU
#    torch_dtype=torch.bfloat16 # What torch dtype to use, defaults to auto
#)

# Load model and tokenizer
#model = AutoModelForImageTextToText.from_pretrained(model_id, **model_kwargs)

model = AutoModelForImageTextToText.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

model.config.use_cache = False

processor = AutoProcessor.from_pretrained(base_model)

/usr/pkg/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-08-15 15:31:45.676895: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755228705.696087  182313 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755228705.700038  182313 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755228705.711615  182313 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755228

In [23]:
print("Visible devices:", accelerator.state.num_processes)
print("Local device:", accelerator.device)

Visible devices: 1
Local device: cuda


In [17]:
from peft import LoraConfig

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.05,
    r=16,
    bias="none",
    target_modules="all-linear",
    task_type="CAUSAL_LM",
    modules_to_save=["lm_head", "embed_tokens"],
)

model = get_peft_model(model, peft_config)

In [18]:
'''
from trl import SFTConfig

args = SFTConfig(
    output_dir="gemma-supplements-small",       # directory to save and repository id
    num_train_epochs=1,                         # number of training epochs
    per_device_train_batch_size=1,              # batch size per device during training
    gradient_accumulation_steps=4,              # number of steps before performing a backward/update pass
    gradient_checkpointing=True,                # use gradient checkpointing to save memory
    optim="adamw_torch_fused",                  # use fused adamw optimizer
    logging_dir="./logs",                       # tensorboard output
    logging_steps=5,                            # log every 5 steps
    save_strategy="epoch",                      # save checkpoint every epoch
    learning_rate=2e-4,                         # learning rate, based on QLoRA paper
    bf16=True,                                  # use bfloat16 precision
    max_grad_norm=0.3,                          # max gradient norm based on QLoRA paper
    warmup_ratio=0.03,                          # warmup ratio based on QLoRA paper
    lr_scheduler_type="constant",               # use constant learning rate scheduler
    push_to_hub=True,                           # push model to hub
    report_to="tensorboard",                    # report metrics to tensorboard
    gradient_checkpointing_kwargs={
        "use_reentrant": False
    },  # use reentrant checkpointing
    dataset_text_field="",                      # need a dummy field for collator
    dataset_kwargs={"skip_prepare_dataset": True},  # important for collator
)
args.remove_unused_columns = False # important for collator
#args.dataloader_pin_memory = False
'''
# Create a data collator to encode text and image pairs
def collate_fn(examples):
    texts, images = [], []
    for example in examples:
        image_inputs = process_vision_info(example["messages"])
        text = processor.apply_chat_template(
            example["messages"], add_generation_prompt=False, tokenize=False
        )
        texts.append(text.strip())
        images.append(image_inputs)

    batch = processor(text=texts, images=images, return_tensors="pt", padding=True)

    labels = batch["input_ids"].clone()
    image_token_id = [
        processor.tokenizer.convert_tokens_to_ids(
            processor.tokenizer.special_tokens_map["boi_token"]
        )
    ]
    labels[labels == processor.tokenizer.pad_token_id] = -100
    labels[labels == image_token_id] = -100
    labels[labels == 262144] = -100
    batch["labels"] = labels

    return batch  # keep on CPU, Accelerator will move to GPU

In [19]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    train_dataset_fmt,
    batch_size=1,
    shuffle=True,
    collate_fn=collate_fn,
)

In [20]:
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4)

In [21]:
model, optimizer, train_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader
)

In [22]:
from torch.nn import CrossEntropyLoss

loss_fn = CrossEntropyLoss()
num_epochs = 1
gradient_accumulation_steps = 4

global_step = 0

model.train()
for epoch in range(num_epochs):
    for step, batch in enumerate(train_dataloader):
        outputs = model(**batch)
        loss = outputs.loss if hasattr(outputs, "loss") else loss_fn(outputs.logits, batch["labels"])

        # Scale loss if using gradient accumulation
        loss = loss / gradient_accumulation_steps
        accelerator.backward(loss)

        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()
            global_step += 1

            # Log to TensorBoard
            writer.add_scalar("train/loss", loss.item() * gradient_accumulation_steps, global_step)

    # Optional: save checkpoint at epoch end
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(f"gemma-supplements-small/epoch{epoch+1}")

OutOfMemoryError: CUDA out of memory. Tried to allocate 1024.00 MiB. GPU 0 has a total capacity of 15.60 GiB of which 589.19 MiB is free. Including non-PyTorch memory, this process has 15.02 GiB memory in use. Of the allocated memory 13.98 GiB is allocated by PyTorch, and 854.08 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
accelerator.wait_for_everyone()
unwrapped_model = accelerator.unwrap_model(model)
unwrapped_model.save_pretrained("gemma-supplements-small/final")
writer.close()